In [2]:
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


XOR 연산은 입력 데이터를 X라고 할 경우 [[0, 0], [0, 1], [1, 0], [1, 1]]과 같은 경우의 수가 있다.
총 4개의 데이터가 각각 2개의 특성을 가지고 있으므로 X는 [4, 2]의 형태로 정의할 수 있다.
입력값에 따른 출력값 Y는 [[0], [1], [1], [0]]이므로 Y는 [4, 1]의 형태로 정의할 수 있다.

In [3]:
X = tf.placeholder(dtype=tf.float32, shape=[4, 2]) # 입력값
Y = tf.placeholder(dtype=tf.float32, shape=[4, 1]) # 출력값

1번째 히든 레이어에서 2개의 입력(X)과 1개의 편향값(B1)을 받아서 2개의 시그모이드 출력(Z)을 다음 레이어로 전달한다.

In [4]:
# 2개의 입력을 받는 2개의 뉴런을 만든다.
# random_normal()은 정규분포로부터 난수값을 반환하고 random_uniform()은 균등 분포로부터의 난수값을 반환한다.
W1 = tf.Variable(tf.random_uniform([2, 2]))
# 각 뉴런은 1개의 편향값을 가진다.
# zeros()는 모든 원소의 값이 0인 텐서를 생성한다.
B1 = tf.Variable(tf.zeros([2]))
# 시그모이드를 거쳐 출력값으로 Z를 리턴한다. => sigmoid(W1 * X + B1)
Z = tf.sigmoid(tf.matmul(X, W1) + B1)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W1))
print(sess.run(B1))
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
print(sess.run(Z, feed_dict={X: train_X}))

[[0.12431252 0.8314338 ]
 [0.49941993 0.12074351]]
[0. 0.]
[[0.5        0.5       ]
 [0.622323   0.5301493 ]
 [0.53103817 0.696658  ]
 [0.65106696 0.72155285]]


2번째 히든 레이어에서 1번째 히든 레이어의 출력값인 Z와 1개의 편향값(B2)을 받아서 1개의 시그모이드를 출력(Y_hat)한다.

In [6]:
# Z를 입력값으로 받는 1개의 뉴런을 만든다.
W2 = tf.Variable(tf.random_uniform([2, 1]))
# 각 뉴런은 1개의 편향값을 가진다.
B2 = tf.Variable(tf.zeros([1]))
# 시그모이드를 거쳐 출력값으로 Y_hat을 리턴한다. => sigmoid(W2 * Z + B1)
Y_hat = tf.sigmoid(tf.matmul(X, W2) + B2)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W2))
print(sess.run(B2))
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
print(sess.run(Z, feed_dict={X: train_X}))
sess.close()

[[0.41412258]
 [0.94159365]]
[0.]
[[0.5        0.5       ]
 [0.5803076  0.61454225]
 [0.5262166  0.69689727]
 [0.60563457 0.78566873]]


손실 함수로 크로스 엔트로피를 사용하고 경사 하강법으로 모델의 매개 변수(가중치, 편향)를 최적화한다.

In [10]:
# 크로스 엔트로피
loss = tf.reduce_mean(
    -1 * (Y * tf.log(Y_hat) + (1 - Y) * tf.log(1.0 - Y_hat))
)
# 경사 하강법
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
# 모델을 학습하기 위한 데이터를 만든다.
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_Y = [[0], [1], [1], [0]]

학습을 진행한다.

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('학습 데이터: {}'.format(train_X))
    print('=' * 80)
    
    for epoch in range(3001):
        sess.run(train_step, feed_dict={X: train_X, Y: train_Y})
        if epoch % 500 == 0:
            print('epoch: {}'.format(epoch))
            print('output\n', sess.run(Y_hat, feed_dict={X:train_X, Y: train_Y}))

학습 데이터: [[0, 0], [0, 1], [1, 0], [1, 1]]
epoch: 0
output
 [[0.49926627]
 [0.51121694]
 [0.54499364]
 [0.5568217 ]]
epoch: 500
output
 [[0.49713728]
 [0.49881303]
 [0.5002885 ]
 [0.5019643 ]]
epoch: 1000
output
 [[0.49959895]
 [0.49990496]
 [0.49996915]
 [0.5002752 ]]
epoch: 1500
output
 [[0.49994382]
 [0.49998978]
 [0.49999258]
 [0.50003856]]
epoch: 2000
output
 [[0.49999213]
 [0.49999872]
 [0.49999884]
 [0.5000054 ]]
epoch: 2500
output
 [[0.4999989 ]
 [0.49999982]
 [0.49999982]
 [0.5000008 ]]
epoch: 3000
output
 [[0.49999985]
 [0.49999997]
 [0.49999997]
 [0.50000006]]


학습 결과를 통해 [0, 0], [1, 1]은 0에 가까운 값을, [0, 1], [1, 0]은 1에 가까운 값을 출력하는 것을 확인할 수 있다.